In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import metnum

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from scipy.stats import pearsonr


In [2]:
def reemplazar_por_metrica(df, variable, resumen, metrica):
    for p in resumen.index:
        df.loc[(df.tipodepropiedad == p) & (df[variable].isnull()), variable] = resumen.loc[p, metrica]
        
def reemplazar_por_moda(df, variable, resumen):
    reemplazar_por_metrica(df, variable, resumen, 'moda')

def limpieza_dataframe(df):
    # revisar si tiene sentido, 500 sin tipodepropiedad
    df.dropna(subset=['tipodepropiedad'], inplace=True)
    
    # columnas que no vamos a usar
    drop_columns = ['centroscomercialescercanos', 'escuelascercanas', 'usosmultiples', 'gimnasio',
               'fecha', 'lng', 'lat', 'idzona', 'id', 'direccion', 'titulo', 'descripcion', 'ciudad']
    if sum(df.columns.isin(drop_columns)) == len(drop_columns):
        df.drop(drop_columns, axis=1, inplace=True)
    
    # garages
    resumen_garages = pd.read_csv('r_garages.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'garages', resumen_garages)
    
    # baños
    resumen_banos = pd.read_csv('r_banos.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'banos', resumen_banos)
    
    # habitaciones
    resumen_habitacion = pd.read_csv('r_habitaciones.csv', index_col='tipodepropiedad')
    
    sin_habitaciones_y_deberian = list(resumen_habitacion[resumen_habitacion.porcentaje_no < 50].index)

    df.loc[(df.habitaciones.isnull())&(~df.tipodepropiedad.isin(sin_habitaciones_y_deberian)), 'habitaciones'] = 0

    for p in sin_habitaciones_y_deberian:
        df.loc[(df.habitaciones.isnull()) & (df.tipodepropiedad == p), 'habitaciones'] = resumen_habitacion.loc[p, 'moda']
    
    # antiguedad
    resumen_antiguedad = pd.read_csv('r_antiguedad.csv', index_col='tipodepropiedad')
    reemplazar_por_moda(df, 'antiguedad', resumen_antiguedad)

    # metrostotales y metroscubiertos
    df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metrostotales'] = df.loc[(~df.metroscubiertos.isnull()) & (df.metrostotales.isnull()), 'metroscubiertos']
    df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metroscubiertos'] = df.loc[(df.metroscubiertos.isnull()) & (~df.metrostotales.isnull()), 'metrostotales']


In [3]:
df = pd.read_csv('../data/train.csv')
limpieza_dataframe(df)

In [4]:
puebla = df[df.provincia=="Puebla"].copy()
puebla.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10421 entries, 26 to 239987
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tipodepropiedad  10421 non-null  object 
 1   provincia        10421 non-null  object 
 2   antiguedad       10421 non-null  float64
 3   habitaciones     10421 non-null  float64
 4   garages          10421 non-null  float64
 5   banos            10421 non-null  float64
 6   metroscubiertos  10421 non-null  float64
 7   metrostotales    10421 non-null  float64
 8   piscina          10421 non-null  float64
 9   precio           10421 non-null  float64
dtypes: float64(8), object(2)
memory usage: 895.6+ KB


In [5]:
puebla['constante'] = 1
data = puebla[['constante', 'metroscubiertos', 'metrostotales', 'antiguedad', 'habitaciones', 'garages', 'banos', 'precio']].copy()

x = data[['constante', 'metroscubiertos', 'metrostotales', 'banos']]
y = data['precio']

x.min(), y.min()

(constante           1.0
 metroscubiertos    15.0
 metrostotales      17.0
 banos               1.0
 dtype: float64,
 310000.0)

## no hay valores menores menores a 1

In [6]:
# x.info()
# y.info()
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)

y_pred = linear_regressor.predict(x)

In [7]:

linear_regressor.coef_

array([     0.        ,   8257.54728203,   4340.71550956, 368090.72099224])

In [8]:
y_pred.min()

-218484.07459045318

## no hay coeficientes negativos, no hay ningun precio negativo y ningun dato del fit negativo, pero trae cosas negativas.... raro

In [9]:
np.argwhere(y_pred < 0)

array([[  628],
       [  721],
       [  762],
       [ 1042],
       [ 1774],
       [ 1920],
       [ 2247],
       [ 2397],
       [ 3144],
       [ 3664],
       [ 3736],
       [ 5969],
       [ 6557],
       [ 6864],
       [ 7702],
       [ 8164],
       [ 8272],
       [ 8940],
       [ 9025],
       [ 9395],
       [ 9546],
       [ 9639],
       [10013]])

In [12]:
print(x.iloc[628, :])
print()
print()
print("Predccion negativa", y_pred[628])
print("Dato calculado a mano", x.iloc[628,:].array @ linear_regressor.coef_)

constante           1.0
metroscubiertos    27.0
metrostotales      27.0
banos               1.0
Name: 13161, dtype: float64


Predccion negativa -92501.4466744765
Dato calculado a mano 708243.8163653788
